In [31]:
import ctypes
import sys

In [2]:
def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [3]:
class Person:
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return f"Person({self.name})"
    def __del__(self): # always called when garbage collector is called
        print(f"__del__ called for {self}...")

In [4]:
p = Person("Alex")
id_p = id(p)
ref_count(id_p)

1

In [5]:
p = None # garbage collector was called

__del__ called for Person(Alex)...


In [6]:
p = Person("Alex")
del p # garbage collector was called

__del__ called for Person(Alex)...


#### we don't know when the garbage collector will be called

In [8]:
class Person:
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return f"Person({self.name})"
    def __del__(self): # always called when garbage collector is called
        print(f"__del__ called for {self}...")
        
    def gen_ex(self):
        raise ValueError("Gen_ex called...")

In [9]:
p = Person("eric")
id_p = id(p)
ref_count(id_p)

1

In [10]:
try:
    p.gen_ex()
except ValueError as ex:
    print(ex)
    

Gen_ex called...


In [11]:
ex # we don't handle the exception after the exception has finished running

NameError: name 'ex' is not defined

a way to have the exception value after the exception run

In [12]:
try:
    p.gen_ex()
except ValueError as ex:
    error = ex 
    print(ex)
    

Gen_ex called...


In [13]:
error

ValueError('Gen_ex called...')

there is another reference to p inside this error (exception object)

In [15]:
ref_count(id_p) 

2

what containes exception object:

In [16]:
dir(error)

['__cause__',
 '__class__',
 '__context__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__suppress_context__',
 '__traceback__',
 'args',
 'with_traceback']

In [17]:
dir(error.__traceback__)

['tb_frame', 'tb_lasti', 'tb_lineno', 'tb_next']

In [18]:
dir(error.__traceback__.tb_frame)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'f_back',
 'f_builtins',
 'f_code',
 'f_globals',
 'f_lasti',
 'f_lineno',
 'f_locals',
 'f_trace',
 'f_trace_lines',
 'f_trace_opcodes']

In [20]:
type(error.__traceback__.tb_frame.f_locals)

dict

In [21]:
for key, value in error.__traceback__.tb_frame.f_locals.items():
    pass

RuntimeError: dictionary changed size during iteration

make a copy to avoid this error

In [22]:
for key, value in error.__traceback__.tb_frame.f_locals.copy().items():
    print(key, value)

__name__ __main__
__doc__ Automatically created module for IPython interactive environment
__package__ None
__loader__ None
__spec__ None
__builtin__ <module 'builtins' (built-in)>
__builtins__ <module 'builtins' (built-in)>
_ih ['', 'import ctypes', 'def ref_count(address):\n    return ctypes.c_long.from_address(address).value', 'class Person:\n    def __init__(self, name):\n        self.name = name\n    def __repr__(self):\n        return f"Person({self.name})"\n    def __del__(self):\n        print(f"__del__ called for {self}...")', 'p = Person("Alex")\nid_p = id(p)\nref_count(id_p)', 'p = None', 'p = Person("Alex")\ndel p # garbage collector was called', 'class Person:\n    def __init__(self, name):\n        self.name = name\n    def __repr__(self):\n        return f"Person({self.name})"\n    def __del__(self): # always called when garbage collector is called\n        print(f"__del__ called for {self}...")\n        \n    def gen_ex(self):\n        raise ValueError("Gen_ex called...

In [23]:
for key, value in error.__traceback__.tb_frame.f_locals.copy().items():
    if isinstance(value, Person):
        print(key, value, id(value))

p Person(eric) 1483097645000


In [24]:
del p
del error

In [25]:
ref_count(id_p)

1

#### all unhandled exception in the __del__ method are ignored by python

In [30]:
class Person:
    def __del__(self):
        raise ValueError("Exception in the __del__ method...")
p = Person()
del p
print("nothing interrupted our code")

nothing interrupted our code


Exception ignored in: <function Person.__del__ at 0x000001594F8D0798>
Traceback (most recent call last):
  File "<ipython-input-30-d73b6a46811c>", line 3, in __del__
ValueError: Exception in the __del__ method...


#### redirecting standart output

In [32]:
sys.stderr, sys.stdout

(<ipykernel.iostream.OutStream at 0x1594d6a5448>,
 <ipykernel.iostream.OutStream at 0x1594d687dc8>)

let's redirect to a file by creating a context manager

In [44]:
class ErrToFile:
    def __init__(self, fname):
        self._fname = fname
        self._current_stderr = sys.stderr
        
    def __enter__(self):
        self._file = open(self._fname, "w")
        sys.stderr = self._file
        
    def __exit__(self, exc_type, exc_value, exc_tb):
        sys.stderr = self._current_stderr
        if self._file:
            self._file.close()
        return False

In [45]:
p = Person()
with ErrToFile('err.txt'):
    del p
    print(100)
print(200)

100
200
